In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder \
.appName("Wordcount_Demo") \
.master("yarn") \
.getOrCreate()

25/07/05 01:28:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
df = spark.read\
.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("gs://my_spark_data/orders.csv")

In [5]:
df.show(10)

+------------+----------------+-------+--------+------+----------+
|    order_id|        customer|product|quantity| price|order_date|
+------------+----------------+-------+--------+------+----------+
|pro_0068ee0e|     Ly Van Dung|   comb|       1| 21000|2025-01-27|
|pro_0068ee0f| Nguyen Duc Tran| Mirror|       3| 80000|2025-01-23|
|pro_0068ee10|      Pham Ba An|   Lamp|       6|120000|2025-02-01|
|pro_0068ee11|       Le Ba Phu| Pencil|       8| 10000|2025-01-21|
|pro_0068ee12|Nguyen Trong Duc|   Bomb|       9|700000|2025-01-31|
|pro_0068ee13|      Le Duc Tai|   pipe|       8|240000|2025-01-21|
|pro_0068ee14|Nguyen Van Hoang|    bed|       3|500000|2025-01-21|
|pro_0068ee15|    Vo Duc Khanh|  glass|       3| 38000|2025-01-12|
|pro_0068ee16|  Tran Trung Mai|   Bomb|       5|700000|2025-01-21|
|pro_0068ee17|   Ly Xuan Huong| Pencil|       3| 10000|2025-01-01|
+------------+----------------+-------+--------+------+----------+
only showing top 10 rows



In [10]:
df.select((df["quantity"] * df["price"]).alias("total")).show(10)

+-------+
|  total|
+-------+
|  21000|
| 240000|
| 720000|
|  80000|
|6300000|
|1920000|
|1500000|
| 114000|
|3500000|
|  30000|
+-------+
only showing top 10 rows



In [15]:
df.agg((sum(df["quantity"] * df["price"])).alias("total revenue")).show()

+-------------+
|total revenue|
+-------------+
|     97321000|
+-------------+



In [21]:
df.groupBy(df["order_date"]) \
  .agg(count("*").alias("total_orders")) \
  .orderBy(df["order_date"]) \
  .show()

+----------+------------+
|order_date|total_orders|
+----------+------------+
|2025-01-01|           2|
|2025-01-02|           6|
|2025-01-03|           1|
|2025-01-04|           2|
|2025-01-05|           3|
|2025-01-06|           1|
|2025-01-10|           8|
|2025-01-11|           4|
|2025-01-12|           5|
|2025-01-13|           1|
|2025-01-14|           3|
|2025-01-15|           4|
|2025-01-16|           5|
|2025-01-17|           5|
|2025-01-18|           3|
|2025-01-19|           6|
|2025-01-20|           2|
|2025-01-21|           5|
|2025-01-22|           1|
|2025-01-23|           4|
+----------+------------+
only showing top 20 rows



In [28]:
df.groupBy(df["order_date"]) \
  .agg(sum(df["quantity"] * df["price"]).alias("total_revenue")) \
  .orderBy((col("total_revenue")).desc())\
  .limit(1)\
  .show()

+----------+-------------+
|order_date|total_revenue|
+----------+-------------+
|2025-01-10|     12683000|
+----------+-------------+



In [30]:
total_by_date = df.groupBy("order_date") \
                  .agg((sum(col("quantity") * col("price"))).alias("total"))
max_total = total_by_date.agg(max("total").alias("max_total")).collect()[0]["max_total"]
total_by_date.filter(col("total") == max_total).show()

+----------+--------+
|order_date|   total|
+----------+--------+
|2025-01-10|12683000|
+----------+--------+



In [31]:
df.groupBy(df["product"])\
.agg(sum(df["quantity"]))\
.show()

+------------+-------------+
|     product|sum(quantity)|
+------------+-------------+
|    keyboard|            6|
|        lock|            6|
|        Lamp|           18|
|         Gun|           19|
|       Shoes|           18|
|      Pencil|           16|
|        pipe|           57|
|         Key|           23|
|        Soap|           24|
|        comb|            7|
|       glass|           34|
|         Bag|            8|
|         bed|            9|
|        bowl|           27|
|         Fan|           33|
|        Bomb|           36|
|       Chair|           20|
|power outlet|           17|
|        Book|           14|
|       cover|           40|
+------------+-------------+
only showing top 20 rows

